In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error      
from torch.utils.data import DataLoader, TensorDataset
from datetime import timedelta


In [8]:
# Load and preprocess data
data = pd.read_csv('Data/Taipei/h23-g1.csv', sep=';', parse_dates=['date_time'])
data.set_index('date_time', inplace=True)

# Skip first day
data = data[data.index >= "2023-03-23 00:00:00"]

# Add hour and minute features
data['hour'] = data.index.hour
data['minute'] = data.index.minute

# Choose relevant columns
target_columns = ['EMove', 'NMove', 'HMove']
features_columns = ['hour', 'minute']

values = data[target_columns + features_columns].values  # Keep hour and minute as well

# Remove values outside 3 standard deviations
for col in target_columns:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]


# Normalize only the target columns (EMove, NMove, HMove) between 0 to 1
scaler = MinMaxScaler()
scaled_target_data = scaler.fit_transform(data[target_columns])

# Combine scaled target data with original non-target features
scaled_data = np.hstack([scaled_target_data, data[features_columns].values])

# Create sequences
def create_sequences(data, seq_length=144):  #144= one day
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(data[i + seq_length][:3])  # Predict EMove, HMove, NMove
    return torch.tensor(np.array(sequences), dtype=torch.float32), torch.tensor(np.array(targets), dtype=torch.float32)

seq_length = 144  # 24-hour sequence for 10-min interval data
X, y = create_sequences(scaled_data, seq_length)

# Time-based split (80% train, 10% validation, 10% test) 
train_size = int(0.8 * len(X))
val_size = int(0.1 * len(X))
test_size = len(X) - train_size - val_size

X_train, X_val, X_test = X[:train_size], X[train_size:train_size + val_size], X[train_size + val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size + val_size], y[train_size + val_size:]

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [14]:
# Define GRU model
class MovementGRU(nn.Module):
    def __init__(self, input_size=5, hidden_size=20, num_layers=2, output_size=3, dropout_rate=0.2): ### hidden size might need to be smaller, and lyers need to be middle
        super(MovementGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        gru_out, _ = self.gru(x)
        gru_out = self.dropout(gru_out)
        return self.fc(gru_out[:, -1, :])  # Predict next 10-min step

model = MovementGRU()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) ### learning rate (uses Adam optimizer)

# Get the last sequence from the dataset (latest available data)
last_sequence = X[-1].unsqueeze(0)  # Shape: (1, seq_length, input_size)
last_timestamp = data.index[-seq_length:].tolist()  # Get the last seq_length timestamps

future_steps = 10  # Predict next 10 time steps (10 min per step)
predicted_timestamps = []
predicted_values = []

best_val_loss = float('inf')  # Track the best validation loss
best_model_state = None        # Save best model parameters
rollback_patience = 2  # Allow 2 epochs of worse validation loss before rollback
rollback_counter = 0

# Training loop
epochs = 15 
for epoch in range(epochs):
    train_loss = 0
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()  # Clears the previous gradients from the optimizer
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    val_loss = 0
    # Move model to evaluation mode
    model.eval()
    y_preds = []
    y_trues = []

    with torch.no_grad():  # No need for gradients

        # Reset predictions before the last epoch
        if epoch == epochs - 1:  # Only keep the final predictions
            predicted_timestamps = []
            predicted_values = []
            # Train the model
            next_time = data.index[-1]  # Start from last real timestamp
            last_sequence = X[-1].unsqueeze(0)  # Start with last known sequence

            for _ in range(future_steps):
                next_pred = model(last_sequence).numpy().flatten().reshape(1, -1)  # Prediction on normalized scale
                # Inverse transform only the target values (EMove, NMove, HMove)
                next_pred_original = scaler.inverse_transform(next_pred[:, :3].reshape(1, -1)).flatten()

                next_time += timedelta(minutes=10)
                predicted_timestamps.append(next_time)
                predicted_values.append(next_pred_original.flatten())  # Store original scale
                
                time_features = np.array([[next_time.hour, next_time.minute]])  # Convert to (1,2) shape
                next_input = np.hstack([next_pred, time_features])  # Now both are (1, X)
                next_input = torch.tensor(next_input, dtype=torch.float32).unsqueeze(0)#.unsqueeze(0)

                last_sequence = torch.cat([last_sequence[:, 1:, :], next_input], dim=1) # uses previous predictions as input for future predictions


        for batch_x, batch_y in val_loader: # Batch_y contains the true values from your validation dataset. output is what your model predicts for batch_x.
            output = model(batch_x)
            y_preds.append(output.numpy())  # Convert tensor to numpy
            y_trues.append(batch_y.numpy()) 
            loss = criterion(output, batch_y)
            val_loss += loss.item()

    # Convert list of arrays to single NumPy array
    y_preds = np.vstack(y_preds)
    y_trues = np.vstack(y_trues)

    # Compute metrics for each feature (E_move, N_move, H_move)
    mae_E = mean_absolute_error(y_trues[:, 0], y_preds[:, 0])
    rmse_E = np.sqrt(mean_squared_error(y_trues[:, 0], y_preds[:, 0]))

    mae_N = mean_absolute_error(y_trues[:, 1], y_preds[:, 1])
    rmse_N = np.sqrt(mean_squared_error(y_trues[:, 1], y_preds[:, 1]))

    mae_H = mean_absolute_error(y_trues[:, 2], y_preds[:, 2])
    rmse_H = np.sqrt(mean_squared_error(y_trues[:, 2], y_preds[:, 2]))

    # Compute total_move (after reversing normalization)
    total_move_true = np.sqrt(y_trues[:, 0]**2 + y_trues[:, 1]**2 + y_trues[:, 2]**2)
    total_move_pred = np.sqrt(y_preds[:, 0]**2 + y_preds[:, 1]**2 + y_preds[:, 2]**2)

    # Compute MAE and RMSE for total_move
    mae_total_move = mean_absolute_error(total_move_true, total_move_pred)
    rmse_total_move = np.sqrt(mean_squared_error(total_move_true, total_move_pred))

    val_loss /= len(val_loader)
    train_loss /= len(train_loader)

    print(f"Epoch {epoch+1}:")
    print(f"Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.6f}")
    print(f"MAE - E_move: {mae_E:.5f}, N_move: {mae_N:.5f}, H_move: {mae_H:.5f}, Total Move: {mae_total_move:.5f}")
    print(f"RMSE - E_move: {rmse_E:.5f}, N_move: {rmse_N:.5f}, H_move: {rmse_H:.5f}, Total Move: {rmse_total_move:.5f} \n")

     # Fine-tuning: Save best model and roll back if needed
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model.state_dict()
        rollback_counter = 0  # Reset counter if validation improves
    else:
        rollback_counter += 1
        if rollback_counter >= rollback_patience:
            print(f"Rolling back to best model after {rollback_patience} bad epochs. Rolling back & reducing learning rate...")
            model.load_state_dict(best_model_state)  # Revert to best model
            for param_group in optimizer.param_groups:
                param_group['lr'] *= 0.5  # Reduce learning rate by half
            rollback_counter = 0  # Reset counter if validation improves

# Final evaluation on the test set
model.eval()
test_loss = 0
y_test_preds = []
y_test_trues = []

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        output = model(batch_x)
        y_test_preds.append(output.numpy())
        y_test_trues.append(batch_y.numpy())

# Convert to NumPy arrays
y_test_preds = np.vstack(y_test_preds)
y_test_trues = np.vstack(y_test_trues)

# Compute metrics on the test set
mae_test_E = mean_absolute_error(y_test_trues[:, 0], y_test_preds[:, 0])
rmse_test_E = np.sqrt(mean_squared_error(y_test_trues[:, 0], y_test_preds[:, 0]))

mae_test_N = mean_absolute_error(y_test_trues[:, 1], y_test_preds[:, 1])
rmse_test_N = np.sqrt(mean_squared_error(y_test_trues[:, 1], y_test_preds[:, 1]))

mae_test_H = mean_absolute_error(y_test_trues[:, 2], y_test_preds[:, 2])
rmse_test_H = np.sqrt(mean_squared_error(y_test_trues[:, 2], y_test_preds[:, 2]))

total_move_test_true = np.sqrt(y_test_trues[:, 0]**2 + y_test_trues[:, 1]**2 + y_test_trues[:, 2]**2)
total_move_test_pred = np.sqrt(y_test_preds[:, 0]**2 + y_test_preds[:, 1]**2 + y_test_preds[:, 2]**2)

mae_test_total_move = mean_absolute_error(total_move_test_true, total_move_test_pred)
rmse_test_total_move = np.sqrt(mean_squared_error(total_move_test_true, total_move_test_pred))

print("Final Test Set Evaluation:")
print(f"Final Test Loss: {test_loss/len(test_loader):.6f}")
print(f"MAE - E_move: {mae_test_E:.5f}, N_move: {mae_test_N:.5f}, H_move: {mae_test_H:.5f}, Total Move: {mae_test_total_move:.5f}")
print(f"RMSE - E_move: {rmse_test_E:.5f}, N_move: {rmse_test_N:.5f}, H_move: {rmse_test_H:.5f}, Total Move: {rmse_test_total_move:.5f}")


predicted_values = np.array(predicted_values) # Convert to NumPy array if it's a list
total_move_future = np.sqrt(predicted_values[:, 0]**2 + predicted_values[:, 1]**2 + predicted_values[:, 2]**2)

# Convert predictions into a DataFrame
predictions_df = pd.DataFrame(predicted_values, columns=['E_move', 'N_move', 'H_move'])
predictions_df['Total_Move'] = total_move_future  
predictions_df['date_time'] = predicted_timestamps

# Print or save predictions
print("Future predictions:")
print(predictions_df)


Epoch 1:
Train Loss: 0.01358, Val Loss: 0.005309
MAE - E_move: 0.02377, N_move: 0.04963, H_move: 0.06641, Total Move: 0.05448
RMSE - E_move: 0.03130, N_move: 0.07879, H_move: 0.09374, Total Move: 0.07931 

Epoch 2:
Train Loss: 0.00731, Val Loss: 0.005418
MAE - E_move: 0.01986, N_move: 0.05314, H_move: 0.06621, Total Move: 0.06027
RMSE - E_move: 0.02739, N_move: 0.08237, H_move: 0.09365, Total Move: 0.08636 

Epoch 3:
Train Loss: 0.00575, Val Loss: 0.003065
MAE - E_move: 0.02016, N_move: 0.04728, H_move: 0.03375, Total Move: 0.03560
RMSE - E_move: 0.02750, N_move: 0.07604, H_move: 0.05189, Total Move: 0.05495 

Epoch 4:
Train Loss: 0.00348, Val Loss: 0.002660
MAE - E_move: 0.02044, N_move: 0.04490, H_move: 0.02406, Total Move: 0.03540
RMSE - E_move: 0.02804, N_move: 0.07282, H_move: 0.04381, Total Move: 0.05471 

Epoch 5:
Train Loss: 0.00308, Val Loss: 0.002655
MAE - E_move: 0.02033, N_move: 0.04527, H_move: 0.02334, Total Move: 0.03651
RMSE - E_move: 0.02793, N_move: 0.07314, H_move: 0

In [13]:
# Check if target values are all zeros
print("Min y_train:", y_train.min().item(), "Max y_train:", y_train.max().item())

# Check if model parameters are updating
before = model.fc.weight.clone().detach()
optimizer.step()
after = model.fc.weight.clone().detach()
print("Weight update difference:", (after - before).abs().sum().item())

# Check if loss is computed correctly
print("Sample model output:", output[0].detach().numpy())
print("Sample batch_y:", batch_y[0].detach().numpy())

Min y_train: 0.0 Max y_train: 1.0
Weight update difference: 0.010810728184878826
Sample model output: [0.47052312 0.5981424  0.5303125 ]
Sample batch_y: [0.35809523 0.9533898  0.57212   ]
